# 17. CBC padding oracle
https://cryptopals.com/sets/3/challenges/17

In [1]:
from Cryptodome.Cipher import AES
from cryptopals.utils import pkcs7_pad, pkcs7_strip, PaddingError
import os
import random

BLOCKSIZE = AES.block_size
KEYSIZE = 32

class Challenge17:
    def __init__(self):
        self.strings = [
            b"MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
            b"MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
            b"MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
            b"MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
            b"MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
            b"MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
            b"MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
            b"MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
            b"MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
            b"MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93" 
        ]
        self.key = os.urandom(KEYSIZE)

    def get_encrypted_data(self, quiet=True) -> (bytes, bytes):
        '''Select at random one of the 10 strings, pad the string out to the 16-byte AES block size and 
        CBC-encrypt it under the key, providing the caller the ciphertext and IV'''
        _string = random.choice(self.strings)
        if not quiet:
            print(f"{_string=}")
        aes_cbc = AES.new(self.key, AES.MODE_CBC) 
        cipher = aes_cbc.encrypt(pkcs7_pad(_string,BLOCKSIZE))
        return cipher, aes_cbc.iv

    def padding_valid(self, cipher: bytes, iv: bytes) -> bool:
        '''Consume the ciphertext produced by the first function, decrypt it, check its padding, and 
        return true or false depending on whether the padding is valid.'''
        aes_cbc = AES.new(self.key, AES.MODE_CBC, iv)
        plaintext = aes_cbc.decrypt(cipher)
        try:
            pkcs7_strip(plaintext)
        except PaddingError:
            return False
        else:
            return True

In [2]:
challenge17 = Challenge17()
cipher, iv = challenge17.get_encrypted_data(quiet=False)
print(f"{cipher=}")
print(f"{iv=}")
print(f"{challenge17.padding_valid(cipher, iv)=}")

_string=b'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g='
cipher=b'\xba\x87\xfd\xc5\xfd\x90\x07\xc2\xffr3\xf0\xb9\xc2i\xf5\xa0$\x18\x99x\x0b\xa5u\xdc\x81Fhw$1\x07e\x9e\xcd(\x8e\xfag\xe0\xaerECen&\x88'
iv=b' \xba\x874\x14\x17O1\x87<\xd2\xa6\xad\xdc}j'
challenge17.padding_valid(cipher, iv)=True


## Attack

Excellent explanation and tutorial at: https://www.nccgroup.com/us/research-blog/cryptopals-exploiting-cbc-padding-oracles/

* By making modifications to the IV, one can predictably modify the plaintext block. Flipping a bit in the IV will flip the corresponding bit in the plaintext.
* By exploiting this properties, one can build a "zeroing" IV that set some (and eventually all) of the plaintext’s bytes to zero.

In [3]:
from cryptopals.utils import bytes_xor

def single_block_attack(block, oracle):
    zeroing_iv = [0]*BLOCKSIZE # zeroing IV starts out nulled
    for pad_value in range(1,BLOCKSIZE+1): # explore all possible padding values to fill all zeroing block
        padding_iv = [pad_value^b for b in zeroing_iv] # xor pad_value with ziv before searching next ziv byte
        for iv_candidate in range(256): # all possible values for IV byte
            padding_iv[-pad_value] = iv_candidate
            iv = bytes(padding_iv)
            if oracle(block, iv): # padding is valid
                # in case pad_value==1, make sure the padding really is of length 1 
                # by changing penultimate block and querying the oracle again
                #if pad_value == 1: 
                #    padding_iv[-2] ^= 1
                #    iv = bytes(padding_iv)
                #    if not oracle(block, iv):
                #        continue  # false positive, keep searching with next pad_value
                break # good pad_value found
        zeroing_iv[-pad_value] = iv_candidate ^ pad_value
    return bytes(zeroing_iv)

def full_attack(iv, cipher, oracle):
    message = iv + cipher
    blocks = [message[i:i+BLOCKSIZE] for i in range(0, len(message), BLOCKSIZE)]
    result = b''
    iv = blocks[0]
    for cipher in blocks[1:]:
        deckey = single_block_attack(cipher,oracle)
        plaintext = bytes_xor(deckey,iv)
        result += plaintext
        iv = cipher
    return result

In [4]:
from base64 import b64decode

challenge17 = Challenge17()
cipher, iv = challenge17.get_encrypted_data()
print(f"{cipher=}")
print(f"{iv=}")

result = full_attack(iv, cipher, challenge17.padding_valid)
print(f"{result=}")

plaintext = b64decode(pkcs7_strip(result).decode())
print(f"{plaintext=}")

cipher=b'\xce\xabh"X\xc2\x879o\x10\x1e\xfag\xa4\xcbaW\x87\xf2\xabbM\x9dq\x11\xaaN\xfaf\xb7\xe9\xfa,!\xe4\xccy\x92t$u\x07\xe42\xbfVB}p\x7f\x08~)\xd1Q\xbc\x85\xfd\x04\xbc\xd5\xbd\xd0\x00\xd5\xed\x80\x13xE\x170o\xab\xb7\xbe\xb6\x87\x0e\xb5'
iv=b'o\xcf\x9en\xc1r\xc8\x1b\xd8\xe4\xb1A\xeda<@'
result=b'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
plaintext=b"000004Burning 'em, if you ain't quick and nimble"


In [5]:
for _ in range(20):
    cipher, iv = challenge17.get_encrypted_data()
    result = full_attack(iv, cipher, challenge17.padding_valid)
    plaintext = b64decode(pkcs7_strip(result).decode())
    print(plaintext)

b'000006And a high hat with a souped up tempo'
b'000005I go crazy when I hear a cymbal'
b'000002Quick to the point, to the point, no faking'
b"000001With the bass kicked in and the Vega's are pumpin'"
b"000007I'm on a roll, it's time to go solo"
b'000000Now that the party is jumping'
b"000001With the bass kicked in and the Vega's are pumpin'"
b"000007I'm on a roll, it's time to go solo"
b"000004Burning 'em, if you ain't quick and nimble"
b'000000Now that the party is jumping'
b'000000Now that the party is jumping'
b'000009ith my rag-top down so my hair can blow'
b"000001With the bass kicked in and the Vega's are pumpin'"
b"000004Burning 'em, if you ain't quick and nimble"
b"000004Burning 'em, if you ain't quick and nimble"
b'000005I go crazy when I hear a cymbal'
b"000007I'm on a roll, it's time to go solo"
b"000003Cooking MC's like a pound of bacon"
b'000005I go crazy when I hear a cymbal'
b"000003Cooking MC's like a pound of bacon"
